# Import Library

In [55]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

Only DS

In [56]:
cwd = os.getcwd()
# 폴더 위치 변경
os.chdir("C:\\Users\hyunj\\Dropbox\\종합설계프로젝트\\중요\\데이터")

data = pd.read_csv('DS.csv', encoding='cp949')
print('DS의 Feature : ' , data.columns)
data.head()

DS의 Feature :  Index(['Case ID', 'Activity', 'Start Timestamp', 'Complete Timestamp',
       'Variant', 'Variant index', 'VESSEL', 'MCHN_ID', 'FULL_EMPTY', 'POD',
       'BLOCK_BAY'],
      dtype='object')


,Case ID,Activity,Start Timestamp,Complete Timestamp,Variant,Variant index,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,AXIU1484082-2,DS-QUAYSIDE,2018-01-31 21:17:52.000,2018-01-31 21:19:24.000,Variant 1,1,ALAP-001/2018,GC112,F,KRPUS,1B-38
1,AXIU1484082-2,DS-MOVE,2018-01-31 21:19:24.000,2018-01-31 21:36:33.000,Variant 1,1,ALAP-001/2018,YT511,F,KRPUS,1B-38
2,AXIU1484082-2,DS-YARDSIDE,2018-01-31 21:36:33.000,2018-01-31 21:38:05.000,Variant 1,1,ALAP-001/2018,TC221,F,KRPUS,1B-38
3,BGBU5060490-3,DS-QUAYSIDE,2018-01-31 20:57:48.000,2018-01-31 20:59:20.000,Variant 1,1,ALAP-001/2018,GC109,F,KRPUS,3F-69
4,BGBU5060490-3,DS-MOVE,2018-01-31 20:59:20.000,2018-01-31 21:11:32.000,Variant 1,1,ALAP-001/2018,YT538,F,KRPUS,3F-69


In [57]:
# 필요없는 열 삭제 & Feature name 변경
data.drop('Variant', axis = 1, inplace = True)
data.drop('Variant index', axis = 1, inplace = True)

data.rename(columns = {'Case ID':'CONTAINER_NO',
                       'Activity':'JOB_TYPE',
                       'Start Timestamp': 'JOB_START_DT',
                       'Complete Timestamp' : 'JOB_END_DT'},
                        inplace=True)
print('DS의 Feature : ' , data.columns)
print('DS의 SHAPE : ', data.shape)
data.head()

DS의 Feature :  Index(['CONTAINER_NO', 'JOB_TYPE', 'JOB_START_DT', 'JOB_END_DT', 'VESSEL',
       'MCHN_ID', 'FULL_EMPTY', 'POD', 'BLOCK_BAY'],
      dtype='object')
DS의 SHAPE :  (148959, 9)


,CONTAINER_NO,JOB_TYPE,JOB_START_DT,JOB_END_DT,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,AXIU1484082-2,DS-QUAYSIDE,2018-01-31 21:17:52.000,2018-01-31 21:19:24.000,ALAP-001/2018,GC112,F,KRPUS,1B-38
1,AXIU1484082-2,DS-MOVE,2018-01-31 21:19:24.000,2018-01-31 21:36:33.000,ALAP-001/2018,YT511,F,KRPUS,1B-38
2,AXIU1484082-2,DS-YARDSIDE,2018-01-31 21:36:33.000,2018-01-31 21:38:05.000,ALAP-001/2018,TC221,F,KRPUS,1B-38
3,BGBU5060490-3,DS-QUAYSIDE,2018-01-31 20:57:48.000,2018-01-31 20:59:20.000,ALAP-001/2018,GC109,F,KRPUS,3F-69
4,BGBU5060490-3,DS-MOVE,2018-01-31 20:59:20.000,2018-01-31 21:11:32.000,ALAP-001/2018,YT538,F,KRPUS,3F-69


In [58]:
# 통합할 데이터 호출
che_data = pd.read_csv('CHE I perspective.csv')
gate_data = pd.read_csv('Gate perspective.csv')
qc_data = pd.read_csv('QC perspective.csv')
vessel_data = pd.read_csv('Vessel perspective.csv')

In [59]:
# 최종 DS csv 파일 - 자세한 것은 파워포인트 참고
temp_df = pd.DataFrame(columns=["CONTAINER_NO", "VESSEL", "MCHN_ID", "FULL_EMPTY", "JOB_TYPE", "POD", "BLOCK_BAY", "JOB_START_DT", "JOB_END_DT",
                                "CHE_ID", "ITV_ID", "QC_ID", "OUTSERVICE", "QUEUE_POSITION", "H_BOX",
                                "OUTVESSEL", "COUNTRY_NEXTPOD", "PORT_NEXTPOD", "TML_OUT_DT"])
print('최종적으로 사용할 데이터 셋의 Column 수는' , len(temp_df.columns))

# temp에서 추가하고 이를 last에서 concat하는 걸로 - 왜냐하면 temp에서 containerid별로 가져올 것이기 때문에, 이렇게 해주는 이유는 정렬하기 위해서 
# (사실 다 추가하고 정렬하면 괜찮지만, 작업 중간 중간에 볼 것이기 때문에 이렇게 함 - 메모리 이슈는 고려하지 않음) 
last_df = temp_df.copy()

최종적으로 사용할 데이터 셋의 Column 수는 19


In [60]:
unique_container_list = list(data['CONTAINER_NO'].unique())
print(len(unique_container_list))

49653


In [61]:
# 데이터 우선 통합 - Container perspective 기준!
temp_df = pd.concat([temp_df,data])

In [62]:
# perspetive를 비교하면서 매치되지 않는 CONTAINER_NO 개수를 기록하기 위한 변수 선
exception_number = 0

for container in tqdm(unique_container_list, desc = "데이터 통합 중") : 
    criteria_df = temp_df[temp_df['CONTAINER_NO'] == container].reset_index(drop=True)
    #! Gate perspective에서 정보 추출 - GO & GI (DS에선 GO!)
    gate_temp_df = gate_data[gate_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    # container perspective에 있는 CONTAINER_NO가 gate_perspective에 없는 경우도 고려해보아야 함
    if len(gate_temp_df) == 0 :
        exception_number += 1
        print(container + "에서 CONTAINER_NO가 매치되지 않는 문제점 발생")
        # 일단 그래도 추가하는 걸로 
    
    else :
        # gate_temp_df의 GO에 대한 정보를 temp_df에 전달 후 
        
    
    
    # CHE I perspective에서 정보 추출 - MOVE
    # QC perspective에서 정보 추출 - QUAYSIDE
    # Vessel perspective에서 정보 추출(Vessel은 DS에서만 사용) - DS_QUAYSIDE
    
    # 각 정보를 전달한 뒤에 last_df에 concat해주기

데이터 통합 중:   3%|▎         | 1273/49653 [00:18<11:40, 69.07it/s]


KeyboardInterrupt: 

In [ ]:
last_df.to_csv('확인.csv', encoding='cp949', index=False)